In [1]:
import pandas as pd
import import_ipynb
import os
import numpy as np
root_path = os.getcwd()
from Preprocessing import *
from Models.models import *

importing Jupyter notebook from Preprocessing.ipynb
importing Jupyter notebook from /home/greca/Sentiment-Analysis-Twitter-PT/Models/models.ipynb


In [2]:
os.chdir(root_path)
data = pd.read_csv('data.csv')
data.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)


In [3]:
data.shape

(785814, 5)

In [4]:
data = data.drop(columns=['id', 'tweet_date', 'query_used'])
data = data.rename({'tweet_text': 'text', 'sentiment': 'target'}, axis=1)
data['target'] = data['target'].apply(lowercase)
data.head()

,text,target
0,@Tixaa23 14 para eu ir :),positivo
1,@drexalvarez O meu like eu já dei na época :),positivo
2,Eu só queria conseguir comer alguma coisa pra ...,positivo
3,:D que lindo dia !,positivo
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",positivo


In [5]:
%%time
data = data.sample(5000)
data = data.reset_index()
data = data.drop(columns=['index'])
data['text'] = preprocessing(data['text'])
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
CPU times: user 673 ms, sys: 92.1 ms, total: 765 ms
Wall time: 3min 37s


In [6]:
data.shape

(4771, 2)

In [7]:
%%time
data['leia'] = data['text'].apply(leia)

CPU times: user 1 s, sys: 0 ns, total: 1 s
Wall time: 1.01 s


In [8]:
%%time
data['textblob'] = data['text'].apply(textblob)

CPU times: user 1h 2min 51s, sys: 24.9 s, total: 1h 3min 16s
Wall time: 1h 3min 25s


In [9]:
%%time
data['oplexion'] = data['text'].apply(oplexion)

CPU times: user 28.9 s, sys: 32 ms, total: 29 s
Wall time: 29 s


In [10]:
%%time
data['sentilex'] = data['text'].apply(sentilex)

CPU times: user 28.4 s, sys: 8.01 ms, total: 28.4 s
Wall time: 28.4 s


In [11]:
data['leia_acertou'] = np.where(data['leia'] == data['target'], True, False)
data['textblob_acertou'] = np.where(data['textblob'] == data['target'], True, False)
data['oplexion_acertou'] = np.where(data['oplexion'] == data['target'], True, False)
data['sentilex_acertou'] = np.where(data['sentilex'] == data['target'], True, False)

In [12]:
leia_accuracy = data['leia_acertou'].sum()/data.shape[0]
textblob_accuracy = data['textblob_acertou'].sum()/data.shape[0]
oplexion_accuracy = data['oplexion_acertou'].sum()/data.shape[0]
sentilex_accuracy = data['sentilex_acertou'].sum()/data.shape[0]

print("LeIA: " + str(leia_accuracy))
print("TextBlob: " + str(textblob_accuracy))
print("OpLexion: " + str(oplexion_accuracy))
print("SentiLex: " + str(sentilex_accuracy))

LeIA: 0.9207713267658771
TextBlob: 0.5680150911758541
OpLexion: 0.8868161810941102
SentiLex: 0.8983441626493398


In [13]:
data = data.drop(columns=['leia_acertou', 'textblob_acertou', 'oplexion_acertou', 'sentilex_acertou'])

In [14]:
data.head(10)

,text,target,leia,textblob,oplexion,sentilex
0,maior tristeza despedir love logo gente não va...,positivo,positivo,positivo,neutro,neutro
1,faz vida cu curioso cara feliz,positivo,positivo,positivo,positivo,positivo
2,migo sou irresponsável cara triste,negativo,negativo,positivo,negativo,negativo
3,harry louis amo vocês cada dia cara triste,negativo,negativo,positivo,negativo,negativo
4,ainda cima baixar episodio drama assim nunca c...,negativo,negativo,positivo,negativo,negativo
5,queria saber dançar rebolado cara triste,negativo,negativo,positivo,negativo,negativo
6,performance abracadabra obra arte cara triste,negativo,negativo,positivo,negativo,negativo
7,coll monica chegou vou chorar cara feliz liber...,positivo,positivo,positivo,positivo,positivo
8,hora queria vocês morassem perto cara triste,negativo,negativo,positivo,negativo,negativo
9,saudades cara triste,negativo,negativo,positivo,negativo,negativo
